- 01_Data.csv 파일을 가져와 아래의 미션을 수행
- Credit_Rank '신용등급'을 예측하는 회귀모델을 만들고자 한다.
    - X : Age / Amount_Month / Term / Gender / Product_Type
    - Y : Credit_Rank -> 결측값 fillna() 중앙값으로 변한
    - 숫자데이터 : 결측처리 + Standard Scaling
    - 문자데이터 : 결측처리 + One Hot Encoding
    - 교차 검증 3회 / DecisionTreeRegressor / Param_grid = {}
    - r2_score (from sklearn.metrics import r2_score)
    - r2_score(Y_train, Y_train_pred) / r2_score(Y_test, Y_test_pred)

In [1]:
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import plotly.express as px
import scipy.stats as stats

mpl.rc('font', family='Malgun Gothic')

In [2]:
df1 = pd.read_csv('01_Data.csv')

In [3]:
X = df1[['Age' , 'Amount_Month' , 'Term' , 'Gender' , 'Product_Type']]
Y = df1['Credit_Rank'].fillna(df1['Credit_Rank'].median())
# 지도학습에서 Y값은 정답지이기 때문에, 결측/이상치 처리를 미리 수행

- Scikit Learn : 
    - model_selection : 데이터와 관련된 함수 (데이터 분할 / 데이터와 관련된 교차검증)
    - preprocessing : 스케일링  및 인코딩과 같은 각 항목의 특성공학을 적용하는 함수
    - impute : 결측값 대치
    - metrics : 평가 관련 함수들

In [4]:
# 학습데이터와 검증데이터를 분할
from sklearn.model_selection import train_test_split
# 파이프라인 
from sklearn.compose import make_column_transformer
from imblearn.pipeline import make_pipeline
# 특성공학 (숫자데이터 처리 + 문자데이터 처리)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
# 교차검증
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
# 평가
from sklearn.metrics import r2_score

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1234)

In [6]:
numeric_pipe = make_pipeline(SimpleImputer(), StandardScaler())
category_pipe = make_pipeline(SimpleImputer(strategy = 'most_frequent'), OneHotEncoder())

In [7]:
numeric_list = X.describe().columns.tolist()
category_list = X.describe(include='object').columns.tolist()
print(numeric_list, category_list)
preprocessing_pipe = make_column_transformer((numeric_pipe, numeric_list), (category_pipe, category_list))
preprocessing_pipe
model_pipe = make_pipeline(preprocessing_pipe, DecisionTreeRegressor())

['Age', 'Amount_Month', 'Term'] ['Gender', 'Product_Type']


In [8]:
grid_model = GridSearchCV(model_pipe, cv=3, param_grid={}, n_jobs=-1)
grid_model.fit(X_train, Y_train)
best_model = grid_model.best_estimator_

In [9]:
Y_train_pred = best_model.predict(X_train)
Y_test_pred = best_model.predict(X_test)

In [10]:
r2_score(Y_train, Y_train_pred)

0.12753941706393723

In [11]:
r2_score(Y_test, Y_test_pred)

-0.0556295159932616

# Mission 10
- '12_Data.csv'
- 아래의 사례들의 대해 어떤 가설 검정 기법을 써야하는지 작성하시오.
- ex. 10대와 20대의 주당방문횟수(정규성X)의 평균의 차이가 있는지 확인해야하는 상황
    -> stats.ranksums()
1. '방송구분'이라는 항목에서 '녹화방송'과 '재방송' 두 집단의 정규성을 띄지 않는 '상품주문수량'의 평균의 차이가 있는지 확인해야 하는 상황
    -> stats.ranksums()
2. 비정규 분포를 따르는 '판매단가'와 '수수료율'의 상관성을 확인해야 하는 상황
    -> stats.spearmanr()
3. '배송방식'과 '매입형태' 간의 연관성이 있는지 확인해야 하는 상황
    -> stats.chi2_contigency()
4. 정규분포를 따르며, 분산이 서로같은 '고객연령'에 대해, 고객의 주소지(3개이상) 별 연령의 평균이 차이가 있는지 확인해야 하는 상황
    -> stats.f_oneway()
5. 정규분포를 따르며, 분산이 서로 다른 '프로그램' 3종류에 따라 '매출액'의 평균이 다른지 확인해야하는 상황
    -> stats.kruskal()

In [13]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats

mpl.rc('font', family='Malgun Gothic')

df1 = pd.read_csv('12_Data.csv')


In [14]:
# 1. 데이터 구조와 타입을 확인하고, 결측값을 각 항목 별 결측값의 개수를 확인하시오.
df1.info()
df1.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26283 entries, 0 to 26282
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   방송Code    26283 non-null  float64
 1   채널        26283 non-null  object 
 2   소요분       26283 non-null  float64
 3   가중분       26283 non-null  float64
 4   방송구분      26283 non-null  object 
 5   프로그램명     26283 non-null  object 
 6   상품ID      26283 non-null  int64  
 7   상품명       26283 non-null  object 
 8   매입과세구분    26283 non-null  object 
 9   상품목표취급금액  26283 non-null  int64  
 10  상품목표주문금액  26283 non-null  int64  
 11  판매단가      26283 non-null  int64  
 12  수수료율      26283 non-null  float64
 13  상품주문수량    26283 non-null  int64  
 14  상품주문금액    26283 non-null  int64  
 15  상품취소수량    26283 non-null  int64  
 16  상품취소금액    26283 non-null  int64  
 17  ARS금액     26283 non-null  int64  
 18  매입형태      26283 non-null  object 
 19  배송방식      26283 non-null  object 
 20  상품소요분     26283 non-null  fl

방송Code      0
채널          0
소요분         0
가중분         0
방송구분        0
프로그램명       0
상품ID        0
상품명         0
매입과세구분      0
상품목표취급금액    0
상품목표주문금액    0
판매단가        0
수수료율        0
상품주문수량      0
상품주문금액      0
상품취소수량      0
상품취소금액      0
ARS금액       0
매입형태        0
배송방식        0
상품소요분       0
상품가중분       0
상품방송순번      0
방송시작시간      0
방송종료시간      0
dtype: int64

In [15]:
# 2. '방송구분'에서 '녹화방송'에 해당하는 데이터만 뽑아서, '상품목표주문금액'의 평균과 중앙값의 차이를 계산하시오
cond1 = (df1['방송구분'] == '녹화방송')
df1.loc[cond1]['상품목표주문금액'].mean() - df1.loc[cond1]['상품목표주문금액'].median()


10473921.794871796

In [16]:
# 3. '상품주문금액'과 '상품목표주문금액'의 차이를 계산하여 '영업이익' 항목을 생성
df1['영업이익'] = df1['상품주문금액'] - df1['상품목표주문금액']
df1

,방송Code,채널,소요분,가중분,방송구분,프로그램명,상품ID,상품명,매입과세구분,상품목표취급금액,...,상품취소금액,ARS금액,매입형태,배송방식,상품소요분,상품가중분,상품방송순번,방송시작시간,방송종료시간,영업이익
0,1.010036e+09,TV,50.0,52.7,녹화방송,재방_의류,10242138,[비지트인뉴욕콜렉션] 퓨어 캐시미어 롱코트,과세,65645345,...,9624136,1000,위탁매입,협력사배송,50.0,52.7,1,2020-01-03T00:10,2020-01-03T01:00,-30939347
1,1.019126e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092003,[한삼인]순홍삼진(50ml*30포)*7박스+쇼7_2,과세,23774849,...,2547305,1000,위탁매입,직택배,30.0,19.1,1,2020-01-03T01:00,2020-01-03T02:00,3040340
2,1.019226e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092743,[단품_한삼인] 순홍삼진(50ml*30포)*1박스,과세,23774849,...,0,0,위탁매입,직택배,30.0,19.1,2,2020-01-03T01:00,2020-01-03T02:00,-29178755
3,1.040337e+09,TV,60.0,19.7,재방송,재방_가공농산,10295865,이상용의 우리밀 크레마롤,과세,24568911,...,1080206,1000,위탁매입,협력사배송,60.0,19.7,1,2020-01-03T02:00,2020-01-03T03:00,-9043773
4,1.021427e+09,TV,60.0,16.1,재방송,재방_신선수산,10113666,국내산 자숙 문어슬라이스 120g*9팩,면세,20131383,...,238512,1000,위탁매입,협력사배송,60.0,16.1,1,2020-01-03T03:00,2020-01-03T04:00,-18459901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26278,1.046263e+13,TV,60.0,94.3,생방송,가전,11179512,■연말특가■제우스커브드 UHD TV 65형,과세,92367433,...,11783638,1000,위탁매입,협력사배송,19.9,32.6,1,2021-11-13T21:45,2021-11-13T22:45,-8303820
26279,1.046263e+13,TV,60.0,94.3,생방송,가전,11179514,2019 제우스 UHD TV 75형,과세,33698544,...,4683636,1000,위탁매입,협력사배송,26.7,41.9,2,2021-11-13T21:45,2021-11-13T22:45,33711022
26280,1.046263e+13,TV,60.0,94.3,생방송,가전,11179504,■연말특가■제우스커브드 UHD TV 55형,과세,17202608,...,3667275,1000,위탁매입,협력사배송,13.4,22.7,3,2021-11-13T21:45,2021-11-13T22:45,15789030
26281,1.020263e+13,TV,60.0,91.6,재방송,재방_건강식품,11152474,[래오이경제] 흑염소진액 70ml x 120포,과세,154250864,...,3879990,1000,위탁매입,협력사배송,60.0,91.6,1,2021-11-13T22:45,2021-11-13T23:45,-7366621


In [20]:
# 4. '방송시작시간' 데이터로부터, 연도/월/요일 을 생성
df1['datetime'] = pd.to_datetime(df1['방송시작시간'])
df1['연도'] = df1['datetime'].dt.year
df1['월'] = df1['datetime'].dt.month
df1['요일'] = df1['datetime'].dt.day_name()

In [21]:
# 5. '방송구분'에 따른 '연도'별 '영업이익'의 증감율을 계산

p1 = df1.pivot_table(index='방송구분', values='영업이익',
                     columns='연도',aggfunc='sum').fillna(0)

p1['증감율'] = (p1[2021] - p1[2020]) / p1[2020]
p1

연도,2020,2021,증감율
방송구분,,,
녹화방송,-1.894549e+08,0.000000e+00,-1.000000
생방송,-3.573885e+10,-8.152800e+10,1.281215
재방송,-1.511756e+10,-1.906222e+10,0.260932


In [22]:
# 6. '방송시작시간'과 '방송종료시간'의 차이를 계산하여 '방송소요시간' 항목을 생성
df1['방송소요시간'] = pd.to_datetime(df1['방송종료시간']) - pd.to_datetime(df1['방송시작시간'])

In [23]:
# 7. '방송소요시간'이 가장 오래걸린 상위 10개의 광고 품목을 확인
df1.sort_values(by='방송소요시간', ascending=False).head(10)['상품명']

2215                   [S]감비천 13주분(감비천액열1주+감비천12주)_3
20257                            예다함 상조(19.06 사은품변경)
18828                                 예다함 상조(4구좌 추가)
19933                            예다함 상조(19.06 사은품변경)
21106                            예다함 상조(19.06 사은품변경)
4133                 닫음[단품]수앤진골드 NEW골든벨 24K 순금 목걸이_3
22795                          중경(임시정부) &  장가계 4박 5일
4134                  닫음[단품]수앤진골드 NEW골든벨 24K 순금 팔찌_3
4132     [특집전] 수앤진NEW골든벨 24K순금 목걸이+팔찌+1.875g 골드바세트_3
15946                                         예다함 상조
Name: 상품명, dtype: object

In [24]:
# 8. '프로그램명' 항목에서 빈도수가 1000건 이상인 데이터만 추출하여, 
# '방송구분' 별 '상품주문금액' 과 '영업이익'의 합을 계산하시오

df_count = pd.DataFrame(df1['프로그램명'].value_counts())
cond1 = (df_count['프로그램명'] >= 1000)

list_1000 = df_count.loc[cond1].index.tolist()

cond1 =df1['프로그램명'].isin(list_1000)
df1.loc[cond1].pivot_table(index='방송구분',  values=['상품주문금액', '영업이익'], aggfunc='sum')


,상품주문금액,영업이익
방송구분,,
생방송,490453552166,-38166117445
재방송,346480899,10929779


In [25]:
# 9. '방송구분'에서 '생방송'과 '녹화방송'의 매출의 차이를 비교하고자 한다.
#     - '영업이익'의 정규성을 확인
#     - '생방송'과 '녹화방송' 집단의 '영업이익'의 평균의 차이가 있는지 가설검정을 수행하십시오.

stats.normaltest(df1['영업이익'])
# p.value < 0.05 / 대립가설 참 / 분포가 정규분포를 따르지 않는다.

cond1 = (df1['방송구분'] == '생방송')
cond2 = (df1['방송구분'] == '녹화방송')

stats.ranksums(df1.loc[cond1]['영업이익'], df1.loc[cond2]['영업이익'])
# P.value > 0.05 / 귀무가설 참 / 두 집단의 영업이익 값의 평균의 차이가 없다.

RanksumsResult(statistic=-1.1432589065314094, pvalue=0.2529311112195203)

In [26]:
# 10. '방송구분'에서 '녹화방송'에 해당하는 데이터만 추출하여, '영업이익'항목에 대해 RobustScaling을 수행하고, 스케일링 된 '영업이익'의 최대값을 계산하시오.

from sklearn.preprocessing import RobustScaler

cond1 = (df1['방송구분'] == '녹화방송')
df2 = df1.loc[cond1]

scaler = RobustScaler()
scaler.fit(df2[['영업이익']])

df_scale = pd.DataFrame(scaler.fit_transform(df2[['영업이익']]))
df_scale.max()


0    2.897498
dtype: float64

In [27]:
# 11. '영업이익'이 양수이면 1 / 음수이면 0으로 변환하여 'Target'항목으로 생성
cond1 = (df1['영업이익'] >= 0)
df1.loc[cond1, 'Target'] = 1
df1.loc[~cond1, 'Target'] = 0
df1['Target']

0        0.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
26278    0.0
26279    1.0
26280    1.0
26281    0.0
26282    0.0
Name: Target, Length: 26283, dtype: float64

In [28]:
# 12. '소요분' / '상품방송순번' / '판매단가' / '수수료율' / 'ARS금액'을 입력했을 때, 'Target'을 분류하는 분류 모델을 생성

X = df1[['소요분','상품방송순번','판매단가','수수료율', 'ARS금액']]
Y = df1['Target']

In [29]:
from sklearn.model_selection import train_test_split
from imblearn.pipeline import make_pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.tree   import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=1234)

model_pipe = make_pipeline(KNNImputer(), StandardScaler()
                            , DecisionTreeClassifier())

hyperparameter_list = {'decisiontreeclassifier__max_depth' : range(5,11)
                        , 'decisiontreeclassifier__min_samples_split' : range(5,11)
                        , 'decisiontreeclassifier__min_samples_leaf' : range(5,11)}

grid_model = GridSearchCV(model_pipe, cv=5, param_grid={}, scoring='f1', n_jobs=-1)
grid_model.fit(X_train, Y_train)

best_model = grid_model.best_estimator_


In [33]:
# 평가
from sklearn.metrics import classification_report

def evaluation_func_class(best_model) :
    Y_train_pred = best_model.predict(X_train)
    Y_test_pred = best_model.predict(X_test)
    print('학습능력평가')
    print(classification_report(Y_train, Y_train_pred))
    print('일반화능력평가')
    print(classification_report(Y_test, Y_test_pred))

In [34]:
evaluation_func_class(best_model)

학습능력평가
              precision    recall  f1-score   support

         0.0       0.82      0.96      0.89     12204
         1.0       0.91      0.66      0.77      7508

    accuracy                           0.85     19712
   macro avg       0.87      0.81      0.83     19712
weighted avg       0.86      0.85      0.84     19712

일반화능력평가
              precision    recall  f1-score   support

         0.0       0.70      0.81      0.75      4129
         1.0       0.56      0.42      0.48      2442

    accuracy                           0.66      6571
   macro avg       0.63      0.62      0.62      6571
weighted avg       0.65      0.66      0.65      6571



In [35]:
# 13. '매입형태'와 '매입과세구분'의 연관성이 있는지 가설검정을 수행

df1_table = pd.crosstab(df1['매입형태'], df1['매입과세구분'])
stats.chi2_contingency(df1_table)
# 통계량 / P.value / 자유도 / 기댓값
# P.value < 0.05 대립가설 참 / '매입형태'와 '매입과세구분' 간의 연관성이 있다.

(186.52831987660014,
 1.8203703751506522e-42,
 1,
 array([[20743.3367576,  4520.6632424],
        [  836.6632424,   182.3367576]]))

# Mission 9

In [ ]:
# 11. 'Y1A'값이 상위 90%이상인 값을 1, 미만인 값을 0로 변환하여 'Target'값으로 선언한 뒤, A1 ~ A6 값을 넣었을 때, Target을 분류하는 분류모델 생성
#    - 8:2 비율로 학습데이터와 검증 데이터 분할 
#    - Min Max Scaling 기법과 Imputation(평균대치), 교차검증 5회 
#    - 가장 성능이 좋은 모델을 찾아, model_target.sav이름으로 저장 
#    - 평가 : 학습능력평가 / 일반화능력평가 

In [37]:
df3 = pd.read_csv('09_Data.csv')

In [38]:
cond1 = (df3['Y1A'] >= df3['Y1A'].quantile(0.09))

df3.loc[cond1, 'Target'] = 1
df3.loc[~cond1, 'Target'] = 0

In [49]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE


In [50]:
X = df3[['A1', 'A2', 'A3' ,'A4', 'A5', 'A6']]
Y = df3['Target']

In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state = 1234)

In [62]:
model_pipe = make_pipeline( SimpleImputer(), MinMaxScaler(), SMOTE(),
                            DecisionTreeClassifier())

hyper_paramter = {'decisiontreeclassifier__max_depth':range(5,11),
                  'decisiontreeclassifier__min_samples_split':range(5,11),
                  'decisiontreeclassifier__class_weight':['balanced']}

grid_model = GridSearchCV(model_pipe, cv=5, scoring='f1', n_jobs = -1, 
                          param_grid=hyper_paramter)
grid_model.fit(X_train, Y_train)
best_model = grid_model.best_estimator_

In [55]:
from sklearn.metrics import classification_report

In [56]:
Y_train_pred = best_model.predict(X_train)
Y_test_pred = best_model.predict(X_test)

In [59]:
print(classification_report(Y_train, Y_train_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        30
         1.0       1.00      1.00      1.00       325

    accuracy                           1.00       355
   macro avg       1.00      1.00      1.00       355
weighted avg       1.00      1.00      1.00       355



In [58]:
print(classification_report(Y_test, Y_test_pred))

'              precision    recall  f1-score   support\n\n         0.0       0.29      0.54      0.38        13\n         1.0       0.94      0.84      0.89       106\n\n    accuracy                           0.81       119\n   macro avg       0.61      0.69      0.63       119\nweighted avg       0.87      0.81      0.83       119\n'